# General aim of this Notebook :

The general aim of this notebook is to find the best hyperparameters for the Fully Connected Neural Network, using GridSearchCV. 

In order to use it, one needs to load the training datas. 

The best hyperparameters found are indicated. 

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split 

# Load the data

In [16]:
data_train_path = 'C:/Users/Flore.MININT-64PJ14C/Documents/GitHub/ML_Project2/results/Split_400_0.8/Combined_All_Train_80.csv'
tx_train = pd.read_csv(data_train_path,sep=",",squeeze=True)
X_train=tx_train.iloc[:, :400]
ytr = pd.read_csv(data_train_path,sep=",",usecols=[400],squeeze=True)
y_train = tf.keras.utils.to_categorical(ytr,num_classes=14)

In [17]:
print(X_train.shape)
print(y_train.shape)


(10747, 400)
(10747, 14)


# Finding the hyperparameters


In [18]:
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [19]:
n_samples, n_features = X_train.shape[0], X_train.shape[1]
n_outputs=14

In [20]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

## Find best batch size and number of epoch, BEST: bs=100 and epoch=10

In [74]:
def create_model():
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=48, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [75]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [1, 10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.290430 using {'batch_size': 100, 'epochs': 10}
0.212102 (0.039150) with: {'batch_size': 1, 'epochs': 10}
0.209001 (0.038168) with: {'batch_size': 1, 'epochs': 50}
0.209581 (0.044404) with: {'batch_size': 1, 'epochs': 100}
0.262900 (0.056330) with: {'batch_size': 10, 'epochs': 10}
0.221120 (0.055011) with: {'batch_size': 10, 'epochs': 50}
0.219665 (0.052009) with: {'batch_size': 10, 'epochs': 100}
0.269104 (0.057271) with: {'batch_size': 20, 'epochs': 10}
0.228678 (0.034826) with: {'batch_size': 20, 'epochs': 50}
0.213169 (0.033853) with: {'batch_size': 20, 'epochs': 100}
0.285777 (0.054195) with: {'batch_size': 40, 'epochs': 10}
0.235370 (0.060106) with: {'batch_size': 40, 'epochs': 50}
0.209195 (0.049535) with: {'batch_size': 40, 'epochs': 100}
0.270753 (0.061441) with: {'batch_size': 60, 'epochs': 10}
0.231198 (0.028733) with: {'batch_size': 60, 'epochs': 50}
0.225287 (0.048227) with: {'batch_size': 60, 'epochs': 100}
0.281314 (0.025681) with: {'batch_size': 80, 'epochs': 10}

## Find best optimizer, BEST : Adagrad

In [21]:
def create_model(optimizer='Adagrad'):
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=400, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [22]:
#create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.268169 using {'optimizer': 'SGD'}
0.268169 (0.032080) with: {'optimizer': 'SGD'}
0.202941 (0.005949) with: {'optimizer': 'RMSprop'}
0.164881 (0.100062) with: {'optimizer': 'Adagrad'}
0.051366 (0.039467) with: {'optimizer': 'Adadelta'}
0.225184 (0.038334) with: {'optimizer': 'Adam'}
0.240258 (0.051223) with: {'optimizer': 'Adamax'}
0.219695 (0.047227) with: {'optimizer': 'Nadam'}


## Find best learning rate, BEST : 0.01

In [25]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=400, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.SGD(lr=learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model



In [26]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.263607 using {'learn_rate': 0.01}
0.216058 (0.070653) with: {'learn_rate': 0.001}
0.263607 (0.041432) with: {'learn_rate': 0.01}
0.207594 (0.009296) with: {'learn_rate': 0.1}
0.200431 (0.022596) with: {'learn_rate': 0.2}
0.190286 (0.018987) with: {'learn_rate': 0.3}


## Find best Network Weight Initialization, BEST: zero

In [27]:
def create_model(init_mode='uniform'):
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=400, activation='relu',kernel_initializer=init_mode))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [28]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.363358 using {'init_mode': 'zero'}
0.280730 (0.018028) with: {'init_mode': 'uniform'}
0.261189 (0.043770) with: {'init_mode': 'lecun_uniform'}
0.264910 (0.047187) with: {'init_mode': 'normal'}
0.363358 (0.030625) with: {'init_mode': 'zero'}
0.265566 (0.034485) with: {'init_mode': 'glorot_normal'}
0.247418 (0.051744) with: {'init_mode': 'glorot_uniform'}
0.292453 (0.034368) with: {'init_mode': 'he_normal'}
0.259237 (0.026453) with: {'init_mode': 'he_uniform'}


## Find best Neuron Activation Function, BEST: 'relu'

In [29]:
def create_model(activation='relu'):
    # create model
    
    model = tf.keras.Sequential()
    model.add(Dense(6, input_dim=400, activation=activation,kernel_initializer='zero'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [30]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.363358 using {'activation': 'softmax'}
0.363358 (0.030625) with: {'activation': 'softmax'}
0.285940 (0.023939) with: {'activation': 'softplus'}
0.284454 (0.025876) with: {'activation': 'softsign'}
0.363358 (0.030625) with: {'activation': 'relu'}
0.261281 (0.023263) with: {'activation': 'tanh'}
0.363358 (0.030625) with: {'activation': 'sigmoid'}
0.351447 (0.015737) with: {'activation': 'hard_sigmoid'}
0.245094 (0.020847) with: {'activation': 'linear'}


## Find best Number of Neurons in the Hidden Layer, BEST: 6

In [31]:
def create_model(neurons=6):
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(neurons, input_dim=400, activation='softmax',kernel_initializer='zero'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [33]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
neurons = [6, 8, 10]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.363358 using {'neurons': 8}
0.362893 (0.030754) with: {'neurons': 6}
0.363358 (0.030625) with: {'neurons': 8}
0.361125 (0.027663) with: {'neurons': 10}
